imports and data uploads
-----------

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter('ignore')
import os
from scipy import stats
import colorcet as cc
import scipy
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
%matplotlib inline

In [2]:
# upload data
cpath = os.getcwd() #assumes mean_df_females is in the same path as the notebook
mean_df = pd.read_csv(cpath+'/mean_df_male.csv')

rearrrange data in structure
----------

In [3]:
# arrange all data in stcuture: 4 phases X 16 mice in each phase X n samples in session
mic = sorted(mean_df['mouse'].unique())
rep = sorted(mean_df['rep'].unique())
msessions=[]
mdbg = []
mmice=[]
mrep = []
for indm, mc in enumerate(mic): # for all mice
    s1=[]
    rep1=[]
    smic1=[]
    m1=[]
    curr=mean_df.loc[(mean_df['mouse']==mc)] # find the current mouse in current phase
    sor=np.argsort(curr['rep'].unique(),axis=0)
    cs=curr['SubjectName'].unique()[sor]
    for inds, ss in enumerate(cs): #all session for a mouse
            syl = curr.loc[curr['SubjectName']==ss]['usage'].values
            area = curr.loc[curr['SubjectName']==ss]['area_mm'].mean()
            s1.append(ss) #session name
            rep1.append(int(curr.loc[curr['SubjectName']==ss]['rep'].unique())) #repetition number
            smic1.append(syl) #syllable usafe
            m1.append(mc) #mouse
    mmice.append(m1) # 16 mics X n sessions 
    mdbg.append(smic1) #syllable usages 16 mice X n sessions X 61 syllables 
    msessions.append(s1) # 16 mice X sessions names
    mrep.append(rep1) # 16 mice X repetitions names

# ----------------------------------------------------------------------- 
# ANALYSIS
# -----------------------------------------------------------------------

In [4]:
# plot/colors definitions
from matplotlib.lines import Line2D
import matplotlib as mpl

# save figures
data_loc=cpath+'/figs/'
try:
    os.mkdir(data_loc)
except FileExistsError:
    # directory already exists
    pass
os.chdir(data_loc)

plt.style.use("seaborn-paper")

rrep_labels = np.unique(np.concatenate(mrep))
mouse_labels = np.unique(np.concatenate(mmice))
max_sylls=len(mdbg[0][1])

## clustering analysis

In [5]:
## for individuals

In [6]:
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
from sklearn.metrics import adjusted_rand_score

# prepare data
frequencies = np.concatenate(mdbg)

#scaling the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(frequencies)

#clustering
nclust=16
kmeans_kwargs = {"init": "random",
                "n_init": 10,
                "max_iter": 300,
                "random_state": 22}

kmeans = KMeans(n_clusters=nclust, **kmeans_kwargs)
kmeans.fit_predict(scaled_features)
#kmeans.fit(scaled_features)

# The actual labels
clust_labels_indv = kmeans.labels_

# explore goodness of fit to true labels of indv mice
_, idx = np.unique(clust_labels_indv, return_index=True) #unique labels by clustering
clust_labels = clust_labels_indv[np.sort(idx)]

# prepare data
mouse_true_labels = np.unique(np.concatenate(mmice)) #unique labels by mice in exp
new_mouse_labels = np.concatenate(mmice.copy()) # all labels of mice in expetiment

#change eacn label in mouse to corresponding label in cluster labels
for i in range(len(mouse_true_labels)): 
    new_mouse_labels[new_mouse_labels==mouse_true_labels[i]] = clust_labels[i]

clust_labels_indv_str =list(map(str,clust_labels_indv)) # turn clusters into string labels

# accuracy of clustering scores:
ari_kmeans = adjusted_rand_score(new_mouse_labels, clust_labels_indv_str)
print('ARI for clustering by individuals for males '+str(ari_kmeans))

ARI for clustering by individuals for males 0.469944533485906
